In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
!pip install einop
from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from torchsummary import summary

there is two method to download the data in the colab from kaggle from the link and from the name

In [9]:
!pip install opendatasets
# !pip install pandas

In [10]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/preatcher/standard-ocr-dataset" , force = True )


100%|██████████| 46.2M/46.2M [00:00<00:00, 258MB/s]

In [11]:
# !unzip data.zip


In [12]:
import os
data = os.listdir("/content/standard-ocr-dataset")
print(data)

['data', 'data2']


In [51]:
data_paths = []
for i in range(0,len(data)):
  data_path = os.path.join("standard-ocr-dataset" , data[i])
  data_paths.append(data_path)
# print (data_paths)

In [52]:
data2 = os.listdir('standard-ocr-dataset/data')
#data1 = os.listdir('standard-ocr-dataset/data')
# print(data2)
#print(data1)

I am takng data2 for my training


In [53]:
dataset_paths = []
for i in range(0,len(data)):
  dataset_path = os.path.join("standard-ocr-dataset/data2" , data2[i])
  dataset_paths.append(dataset_path)
# print (dataset_paths)

In [55]:
# from re import I

def data_tensor(datasetpath ):
  Dataset = []
  lables_dict = {}
  # print(lables_dict)
  dataset = os.listdir(datasetpath)
  data_f = sorted(dataset)
  for i in range(len(data_f)):
    lables_dict[data_f[i]] = i
    data_f_path = os.path.join(datasetpath, data_f[i])
    data_img = os.listdir(data_f_path)
    for j in range(len(data_img)):
      data_img_path = os.path.join(data_f_path, data_img[j])
      img = Image.open(data_img_path)
      transform = Compose([Resize((64, 64)), ToTensor()])
      im = transform(img)
      rgb_img= im.unsqueeze(0) # add batch dim
      rgb_img = im.expand(1,3,64,64)
      Dataset.append([rgb_img, np.eye(36)[i]])

  return Dataset
# DAta ='/content/standard-ocr-dataset/data/testing_data'
# testing_data= data_tensor(DAta)
# # print(Data)
# print(type(Data))

In [56]:
DAta ='/content/standard-ocr-dataset/data/testing_data'
testing_data= data_tensor(DAta)

In [57]:
# for i in testing_dataset:
  # print(i.shape)


In [58]:
Training_dataset_path = 'standard-ocr-dataset/data/training_data'
training_data = data_tensor(Training_dataset_path)
# for i in training_dataset:

  # print(i.shape)

In [59]:
# class_names = sorted(os.listdir(Training_dataset_path))
# print(class_names)


In [60]:
patch_list = []
# print(type(Data))
n = len(training_data)
for i in range(n):
  patch_size = 8 # 16 pixel
  # print(type(training_data[i][0]))
  pathes = rearrange(training_data[i][0], 'b c (h s1) (w s2) -> b (h w) (s1 s2 c)', s1=patch_size,
                   s2=patch_size)
  patch_list.append(pathes)
  # print(patch_list[i])

# del n


In [67]:
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels: int = 3, patch_size: int = 8, emb_size: int = 64, img_size: int = 64):
        self.patch_size = patch_size
        super().__init__()
        self.projection = nn.Sequential(
            # using a conv layer instead of a linear one -> performance gains
            nn.Conv2d(in_channels, emb_size, kernel_size=patch_size, stride=patch_size),
            Rearrange('b e (h) (w) -> b (h w) e'),
        )
        self.cls_token = nn.Parameter(torch.randn(1,1, emb_size))
        self.positions = nn.Parameter(torch.randn((img_size // patch_size) **2 + 1, emb_size))


    def forward(self, x: Tensor) -> Tensor:
        b, _, _, _ = x.shape
        x = self.projection(x)
        cls_tokens = repeat(self.cls_token, '() n e -> b n e', b=b)
        # prepend the cls token to the input
        x = torch.cat([cls_tokens, x], dim=1)
        # add position embedding
        x += self.positions
        return x
for i in range(int(len(training_data))):
  PatchEmbedding()(training_data[i][0])

In [72]:
class MultiHeadAttention(nn.Module):
    def __init__(self, emb_size: int = 64, num_heads: int = 8, dropout: float = 0):
        super().__init__()
        self.emb_size = emb_size
        self.num_heads = num_heads
        # fuse the queries, keys and values in one matrix
        self.qkv = nn.Linear(emb_size, emb_size * 3)
        self.att_drop = nn.Dropout(dropout)
        self.projection = nn.Linear(emb_size, emb_size)

    def forward(self, x : Tensor, mask: Tensor = None) -> Tensor:
        # split keys, queries and values in num_heads
        qkv = rearrange(self.qkv(x), "b n (h d qkv) -> (qkv) b h n d", h=self.num_heads, qkv=3)
        queries, keys, values = qkv[0], qkv[1], qkv[2]
        # sum up over the last axis
        energy = torch.einsum('bhqd, bhkd -> bhqk', queries, keys) # batch, num_heads, query_len, key_len
        if mask is not None:
            fill_value = torch.finfo(torch.float32).min
            energy.mask_fill(~mask, fill_value)

        scaling = self.emb_size ** (1/2)
        att = F.softmax(energy, dim=-1) / scaling
        att = self.att_drop(att)
        # sum up over the third axis
        out = torch.einsum('bhal, bhlv -> bhav ', att, values)
        out = rearrange(out, "b h n d -> b n (h d)")
        out = self.projection(out)
        return out

for i in range(int(len(training_data))):
  patches_embedded = PatchEmbedding()(training_data[i][0])
  MultiHeadAttention()(patches_embedded)

In [73]:
class ResidualAdd(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        res = x
        x = self.fn(x, **kwargs)
        x += res
        return x

In [74]:
class FeedForwardBlock(nn.Sequential):
    def __init__(self, emb_size: int, expansion: int = 4, drop_p: float = 0.):
        super().__init__(
            nn.Linear(emb_size, expansion * emb_size),
            nn.GELU(),
            nn.Dropout(drop_p),
            nn.Linear(expansion * emb_size, emb_size),
        )

In [75]:
class TransformerEncoderBlock(nn.Sequential):
    def __init__(self,
                 emb_size: int = 64,
                 drop_p: float = 0.,
                 forward_expansion: int = 4,
                 forward_drop_p: float = 0.,
                 ** kwargs):
        super().__init__(
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                MultiHeadAttention(emb_size, **kwargs),
                nn.Dropout(drop_p)
            )),
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                FeedForwardBlock(
                    emb_size, expansion=forward_expansion, drop_p=forward_drop_p),
                nn.Dropout(drop_p)
            )
            ))

In [76]:
for i in range(int(len(training_data))):
  patches_embedded = PatchEmbedding()(training_data[i][0])
  TransformerEncoderBlock()(patches_embedded).shape

In [77]:
class TransformerEncoder(nn.Sequential):
    def __init__(self, depth: int = 12, **kwargs):
        super().__init__(*[TransformerEncoderBlock(**kwargs) for _ in range(depth)])


In [78]:
class ClassificationHead(nn.Sequential):
    def __init__(self, emb_size: int = 128, n_classes: int = 1000):
        super().__init__(
            Reduce('b n e -> b e', reduction='mean'),
            nn.LayerNorm(emb_size),
            nn.Linear(emb_size, n_classes))

In [79]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from tqdm import tqdm

In [81]:
class ViT(nn.Sequential):
    def __init__(self,
                in_channels: int = 3,
                patch_size: int = 16,
                emb_size: int = 64,
                img_size: int = 64,
                depth: int = 12,
                n_classes: int = 36,
                **kwargs):
        super().__init__(
            PatchEmbedding(in_channels, patch_size, emb_size, img_size),
            TransformerEncoder(depth, emb_size=emb_size, **kwargs),
            ClassificationHead(emb_size, n_classes)
        )
    def save(self, path="best_model.pth"):
        torch.save(self.state_dict(), path)

    def load(self, path="best_model.pth"):
        self.load_state_dict(torch.load(path))
        self.eval()

    def fit(
        self,
        train_loader,
        optimizer=Adam,
        loss_function=CrossEntropyLoss(),
        learning_rate=0.001,
        epochs=5,
        tqdm_show=False,
    ):
        min_loss = float("inf")
        self.optimizer = optimizer(self.parameters(), lr=learning_rate)
        self.loss_func = loss_function
        self.epochs = epochs

        if tqdm_show:
            for epoch in tqdm(range(self.epochs)):
                for batch in tqdm(train_loader):
                    train_X, train_y = batch
                    pred_train_y = self.forward(train_X)

                    loss = self.loss_func(pred_train_y, train_y)

                    # self.zero_grad()
                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()

                print(f"epoch={epoch+1}: loss={loss}")
                # if loss < min_loss:
                #     self.save()
                #     min_loss = loss
        else:
            for epoch in range(self.epochs):
                for batch in train_loader:
                    train_X, train_y = batch
                    pred_train_y = self.forward(train_X)

                    loss = self.loss_func(pred_train_y, train_y)

                    # self.zero_grad()
                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()

                print(f"epoch={epoch+1}: loss={loss}")
                # if loss < min_loss:
                #     self.save()
                #     min_loss = loss

        self.save()

    def test(self, test_loader):
        self.load()

        total = len(test_loader.dataset)
        correct = 0

        for batch in test_loader:
            test_X, test_y = batch
            pred_test_y = self.forward(test_X)

            for each in range(len(test_y)):
                pred = torch.argmax(pred_test_y, 1)[each]
                true = torch.argmax(test_y, 1)[each]
                if pred == true:
                    correct += 1

        print(f"accuracy results: {correct}/{total} => {correct/total}")
        return correct / total

In [82]:
BATCH_SIZE = 64

In [83]:
train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=BATCH_SIZE, shuffle=True)

In [84]:
NUM_CHANNELS = 1

In [90]:
ViT.fit(train_dataloader, Adam, epochs=1, learning_rate=0.01, tqdm_show=True)

In [ ]:
ViT.test(train_dataloader)